# Exercises

In this section we have two exercises:
1. Implement the polynomial kernel.
2. Implement the multiclass C-SVM.

## Polynomial kernel

You need to extend the ``build_kernel`` function and implement the polynomial kernel if the ``kernel_type`` is set to 'poly'. The equation that needs to be implemented:
\begin{equation}
K=(X^{T}*Y)^{d}.
\end{equation}

In [15]:
import numpy as np

%store -r train_data_set
%store -r train_labels
%store -r test_data_set
%store -r test_labels

In [16]:
#according to https://en.wikipedia.org/wiki/Polynomial_kernel "The most common degree is d = 2 (quadratic)."
def build_kernel(data_set, kernel_type='linear', d = 2):
    kernel = np.dot(data_set, data_set.T)
    if kernel_type == 'rbf':
        sigma = 1.0
        objects_count = len(data_set)
        b = np.ones((len(data_set), 1))
        kernel -= 0.5 * (np.dot((np.diag(kernel)*np.ones((1, objects_count))).T, b.T)
                         + np.dot(b, (np.diag(kernel) * np.ones((1, objects_count))).T.T))
        kernel = np.exp(kernel / (2. * sigma ** 2))
    elif kernel_type == 'poly':
        kernel = np.matrix(kernel)**d # put your code here
        kernel = np.array(kernel)
    return kernel

In [18]:
build_kernel(train_data_set, 'poly')

array([[229382.4603, 169340.9093, 194767.5298, ..., 201990.1791,
        171105.3501, 250841.2972],
       [169340.9093, 125642.4972, 144493.9497, ..., 149901.869 ,
        126973.778 , 184985.3786],
       [194767.5298, 144493.9497, 166174.3575, ..., 172392.2437,
        146024.6959, 212765.6728],
       ...,
       [201990.1791, 149901.869 , 172392.2437, ..., 178847.5088,
        151491.2048, 220639.497 ],
       [171105.3501, 126973.778 , 146024.6959, ..., 151491.2048,
        128320.1897, 186906.204 ],
       [250841.2972, 184985.3786, 212765.6728, ..., 220639.497 ,
        186906.204 , 274370.3014]])

## Implement a multiclass C-SVM

Use the classification method that we used in notebook 7.3 and IRIS dataset to build a multiclass C-SVM classifier. Most implementation is about a function that will return the proper data set that need to be used for the prediction. You need to implement:
- ``choose_set_for_label``
- ``get_labels_count``

In [20]:
def choose_set_for_label(data_set, label, all_labels):
    # your code here
    labels = np.deepcopy(all_labels)
    labels[labels!=label] = -1
    labels[labels==label] =  1
    train_data_set, test_data_set, train_labels, test_labels = train_test_split(
    data_set, labels, test_size=0.2, random_state=15)
    return train_data_set, test_data_set, train_labels, test_labels

In [21]:
def get_labels_count(data_set):
    return len(data_set)

Use the code that we have implemented earlier:

In [22]:
def train(train_data_set, train_labels, kernel_type='linear', C=10, threshold=1e-5):
    kernel = build_kernel(train_data_set, kernel_type=kernel_type)

    P = train_labels * train_labels.transpose() * kernel
    q = -np.ones((objects_count, 1))
    G = np.concatenate((np.eye(objects_count), -np.eye(objects_count)))
    h = np.concatenate((C * np.ones((objects_count, 1)), np.zeros((objects_count, 1))))

    A = train_labels.reshape(1, objects_count)
    A = A.astype(float)
    b = 0.0

    sol = cvxopt.solvers.qp(cvxopt.matrix(P), cvxopt.matrix(q), cvxopt.matrix(G), cvxopt.matrix(h), cvxopt.matrix(A), cvxopt.matrix(b))

    lambdas = np.array(sol['x'])

    support_vectors_id = np.where(lambdas > threshold)[0]
    vector_number = len(support_vectors_id)
    support_vectors = train_data_set[support_vectors_id, :]

    lambdas = lambdas[support_vectors_id]
    targets = train_labels[support_vectors_id]

    b = np.sum(targets)
    for n in range(vector_number):
        b -= np.sum(lambdas * targets * np.reshape(kernel[support_vectors_id[n], support_vectors_id], (vector_number, 1)))
    b /= len(lambdas)

    return lambdas, support_vectors, support_vectors_id, b, targets, vector_number

def build_kernel(data_set, kernel_type='linear'):
    kernel = np.dot(data_set, data_set.T)
    if kernel_type == 'rbf':
        sigma = 1.0
        objects_count = len(data_set)
        b = np.ones((len(data_set), 1))
        kernel -= 0.5 * (np.dot((np.diag(kernel)*np.ones((1, objects_count))).T, b.T)
                         + np.dot(b, (np.diag(kernel) * np.ones((1, objects_count))).T.T))
        kernel = np.exp(kernel / (2. * sigma ** 2))
    return kernel

def classify_rbf(test_data_set, train_data_set, lambdas, targets, b, vector_number, support_vectors, support_vectors_id):
    kernel = np.dot(test_data_set, support_vectors.T)
    sigma = 1.0
    #K = np.dot(test_data_set, support_vectors.T)
    #kernel = build_kernel(train_data_set, kernel_type='rbf')
    c = (1. / sigma * np.sum(test_data_set ** 2, axis=1) * np.ones((1, np.shape(test_data_set)[0]))).T
    c = np.dot(c, np.ones((1, np.shape(kernel)[1])))
    #aa = np.dot((np.diag(K)*np.ones((1,len(test_data_set)))).T[support_vectors_id], np.ones((1, np.shape(K)[0]))).T
    sv = (np.diag(np.dot(train_data_set, train_data_set.T))*np.ones((1,len(train_data_set)))).T[support_vectors_id]
    aa = np.dot(sv,np.ones((1,np.shape(kernel)[0]))).T
    kernel = kernel - 0.5 * c - 0.5 * aa
    kernel = np.exp(kernel / (2. * sigma ** 2))

    y = np.zeros((np.shape(test_data_set)[0], 1))
    for j in range(np.shape(test_data_set)[0]):
        for i in range(vector_number):
            y[j] += lambdas[i] * targets[i] * kernel[j, i]
        y[j] += b
    return np.sign(y)

In [23]:
# modify this part 
from sklearn.datasets import load_iris
import numpy as np
from sklearn.model_selection import train_test_split
import cvxopt

iris = load_iris()
data_set = iris.data
labels = iris.target

In [6]:
from sklearn.metrics import accuracy_score

for label in np.unique(labels):
    #data_set_copy = np.copy(data_set)
    train_data_set, test_data_set, train_labels, test_labels = choose_set_for_label(data_set,label,labels)
    objects_count = get_labels_count(train_data_set)

    lambdas, support_vectors, support_vectors_id, b, targets, vector_number = train(train_data_set, train_labels, kernel_type='rbf')
    predicted = classify_rbf(test_data_set, train_data_set, lambdas, targets, b, vector_number, support_vectors, support_vectors_id)
    predicted = list(predicted.astype(int))
    print(accuracy_score(predicted, test_labels))

     pcost       dcost       gap    pres   dres
 0:  1.1726e+02 -1.7992e+03  4e+03  2e-01  2e-15
 1:  7.7967e+01 -1.6973e+02  3e+02  5e-03  2e-15
 2:  1.0169e+01 -2.2323e+01  3e+01  9e-16  3e-15
 3: -4.9607e-01 -4.9938e+00  4e+00  2e-16  2e-15
 4: -1.4344e+00 -2.5218e+00  1e+00  2e-16  5e-16
 5: -1.6897e+00 -2.1674e+00  5e-01  2e-16  2e-16
 6: -1.8078e+00 -2.0073e+00  2e-01  1e-16  2e-16
 7: -1.8464e+00 -1.9763e+00  1e-01  3e-16  2e-16
 8: -1.8825e+00 -1.9020e+00  2e-02  2e-16  2e-16
 9: -1.8902e+00 -1.8906e+00  4e-04  2e-16  2e-16
10: -1.8904e+00 -1.8904e+00  5e-06  5e-16  2e-16
11: -1.8904e+00 -1.8904e+00  5e-08  6e-16  2e-16
Optimal solution found.
0.26666666666666666
     pcost       dcost       gap    pres   dres
 0:  1.1410e+02 -2.4247e+03  5e+03  2e-01  2e-15
 1:  9.0959e+01 -2.2448e+02  4e+02  6e-03  3e-15
 2:  1.3024e+01 -2.6284e+01  4e+01  4e-05  4e-15
 3: -1.1088e-01 -5.5940e+00  5e+00  2e-16  2e-15
 4: -1.3519e+00 -2.4130e+00  1e+00  2e-16  6e-16
 5: -1.5743e+00 -2.0232e+00